In [2]:
import numpy as np
from itertools import product, combinations
import warnings
import matplotlib.pyplot as plt
import networkx as nx
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import random
import pprint
pp = pprint.PrettyPrinter(depth=4)

# Qiskit imports for simulation and circuit construction
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.quantum_info import Statevector, partial_trace

# Custom module imports 
from initialization.state_preparation.gates import WeightedUnaryEncoding
from initialization.calculate_w import get_optimal_w, calculate_w
from initialization.construct_A import construct_A_matrix
from dicke_state_preparation.gates import UnkStatePreparation
from decoding.gates import GJEGate
from utils import *

warnings.filterwarnings("ignore")

In [3]:
# Parameters for graph generation and search
num_nodes = 6              # Number of nodes (variables)
num_edges = 6              # Number of XOR constraints (edges)
target_max_solutions = None   # Desired number of optimal assignments
max_iter = 1000            # Maximum search iterations
seed = 666            # Random seed for reproducibility

# Flag to decide if statevector plots should be shown (only for small problems)
show_sv = (num_nodes * num_edges < 50)

# Search for a graph with exactly the target number of max solutions
result = find_graph_with_target_max_solutions(num_nodes, num_edges, target_max_solutions, seed, max_iter)
G, best_assignments, max_satisfied, iterations = result

B, v = get_max_xorsat_matrix(G, v=None)  # v defaults to all ones if not provided


In [24]:
n = len(B[0])  # Number of qubits corresponding to constraints
m, p, r, ell = len(B), 2, 1, 2  # Additional parameters for state preparation

# Compute optimal weight values for the initialization
w, _ = get_optimal_w(m, n, ell, p, r)
print("Optimal W_k value: ", w.tolist())

INFO:utils:Function 'get_optimal_w' executed in 0.000997 seconds.


Optimal W_k value:  [0.0, 0.7071067811865475, 0.7071067811865477, 0.0]


In [25]:
# Build the initialization circuit using Weighted Unary Encoding
init_qregs = QuantumRegister(m, name='k')
initialize_circuit = QuantumCircuit(init_qregs)
WUE_Gate = WeightedUnaryEncoding(m, w)
initialize_circuit.append(WUE_Gate, range(m))
initialize_circuit.decompose().decompose().draw()# .count_ops()


┌──────────┐                                                          »
k_0: ┤ R(π,π/2) ├──■────────────────■──────────────────────────────────────»
     ├─────────┬┘┌─┴─┐┌──────────┐┌─┴─┐                                    »
k_1: ┤ Ry(π/4) ├─┤ X ├┤ Ry(-π/4) ├┤ X ├──■─────────────■───────────────────»
     └┬───────┬┘ └───┘└──────────┘└───┘┌─┴─┐┌───────┐┌─┴─┐                 »
k_2: ─┤ Ry(0) ├────────────────────────┤ X ├┤ Ry(0) ├┤ X ├──■──────────────»
     ┌┴───────┴┐                       └───┘└───────┘└───┘┌─┴─┐┌──────────┐»
k_3: ┤ Ry(π/2) ├──────────────────────────────────────────┤ X ├┤ Ry(-π/2) ├»
     └─────────┘                                          └───┘└──────────┘»
k_4: ──────────────────────────────────────────────────────────────────────»
                                                                           »
k_5: ──────────────────────────────────────────────────────────────────────»
                                                                           »
«          
«k_0: ─────
«          
«k_1: ─────
«          
«k_2: ──■──
«     ┌─┴─┐
«k_3: ┤ X ├
«     └───┘
«k_4: ─────
«          
«k_5: ─────
«

In [30]:
# Build the initialization circuit using Weighted Unary Encoding
from qiskit.circuit.library import RYGate
init_qregs = QuantumRegister(m, name='k')
test_qc = QuantumCircuit(init_qregs)
ccry = RYGate(0.5).control(num_ctrl_qubits=2, ctrl_state="11")
test_qc.append(ccry, [0, 1, 2])
test_qc.decompose().draw()

┌───┐                  ┌───┐                 
k_0: ─────────────────┤ X ├────────■─────────┤ X ├────────■────────
                      └─┬─┘        │         └─┬─┘        │        
k_1: ────────■──────────■──────────┼───────────■──────────┼────────
     ┌───────┴───────┐     ┌───────┴────────┐     ┌───────┴───────┐
k_2: ┤ U(0.25,0,0,0) ├─────┤ U(-0.25,0,0,0) ├─────┤ U(0.25,0,0,0) ├
     └───────────────┘     └────────────────┘     └───────────────┘
k_3: ──────────────────────────────────────────────────────────────
                                                                   
k_4: ──────────────────────────────────────────────────────────────
                                                                   
k_5: ──────────────────────────────────────────────────────────────

In [26]:
transpile(initialize_circuit, basis_gates=['ry', 'cx'])

INFO:qiskit.passmanager.base_tasks:Pass: ContainsInstruction - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: UnitarySynthesis - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: HighLevelSynthesis - 2.03276 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: BasisTranslator - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: ElidePermutations - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: RemoveDiagonalGatesBeforeMeasure - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: RemoveIdentityEquivalent - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: InverseCancellation - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: CommutativeCancellation - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: ConsolidateBlocks - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Split2QUnitaries - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: UnitarySynthesis - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: HighLevelSynthesis - 0.59342 (ms)
INFO:qisk

TranspilerError: 'Unable to translate the operations in the circuit: ["ry", "u", "cx"] to the backend\'s (or manually specified) target basis: {"ry", "store", "snapshot", "cx", "if_else", "for_loop", "switch_case", "while_loop", "reset", "delay", "barrier", "measure"}. This likely means the target basis is not universal or there are additional equivalence rules needed in the EquivalenceLibrary being used. For more details on this error see: https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.passes. BasisTranslator#translation-errors'